In [1]:
import yfinance as yf
df=yf.download("AAPL",period="5y")

/tmp/ipython-input-2795944913.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download("AAPL",period="5y")
[*********************100%***********************]  1 of 1 completed


In [2]:
pip install yfinance torch torchvision torchaudio scikit-learn matplotlib tqdm

In [3]:
import yfinance as yf
df=yf.download("AAPL",period="5y",interval="1d")
df=df.dropna()
df.head()

/tmp/ipython-input-3568428409.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download("AAPL",period="5y",interval="1d")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-11-25,112.949669,113.650556,112.112499,112.482416,76499200
2020-11-27,113.494812,114.370921,113.134639,113.475346,46691300
2020-11-30,115.889503,117.758530,113.708965,113.864721,169410200
2020-12-01,119.462067,120.192156,116.824012,117.797464,127728200
2020-12-02,119.812531,120.094833,117.680667,118.780666,89004200


In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler

seq_len=30
#update features to use correct MultiIndex column names
features=[('Close','AAPL'),('Volume','AAPL')]

def build_sequences(df):
  scaler=StandardScaler()
  #Access columns using correct MultiIndex names
  scaled=scaler.fit_transform(df[features])

  x,y=[],[]
  for i in range(len(scaled)-seq_len-1):
    x.append(scaled[i:i+seq_len])
    #target:whether next day's close is higher
    #Access 'close' column using its correct multiIndex name
    next_up=1 if df[('Close','AAPL')].iloc[i+seq_len+1]>df[('Close','AAPL')].iloc[i+seq_len] else 0
    y.append(next_up)
  return np.array(x),np.array(y),scaler

x,y,scaler=build_sequences(df)
x.shape,y.shape

((1224, 30, 2), (1224,))

In [6]:
n=len(x)
train_end=int(0.7*n)
val_end=int(0.85*n)

x_train,y_train=x[:train_end],y[:train_end]
x_val,y_val=x[train_end:val_end],y[train_end:val_end]
x_test,y_test=x[val_end:],y[val_end:]

In [7]:
from torch.nn.modules import rnn
import torch
import torch.nn as nn

class SeqModel(nn.Module):
  def __init__(self,input_dim,hidden_dim=64,rnn_type="lstm"):
    super().__init__()
    if rnn_type=="lstm":
      self.rnn=nn.LSTM(input_dim,hidden_dim,batch_first=True)
    elif rnn_type=="gru":
      self.rnn=nn.GRU(input_dim,hidden_dim,batch_first=True)
    else:
      self.rnn=nn.RNN(input_dim,hidden_dim,batch_first=True)

    self.fc=nn.Linear(hidden_dim,1)

  def forward(self,x):
    out,_=self.rnn(x)
    last=out[:,-1,:]
    return self.fc(last)

In [8]:
from torch.optim import optimizer
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset
device="cuda" if torch.cuda.is_available() else "cpu"
def train_model(x_train,y_train,x_val,y_val,rnn_type="lstm"):
  x_train_t=torch.tensor(x_train,dtype=torch.float32).to(device)
  y_train_t=torch.tensor(y_train,dtype=torch.float32).to(device)
  x_val_t=torch.tensor(x_val,dtype=torch.float32).to(device)
  y_val_t=torch.tensor(y_val,dtype=torch.float32).to(device)
  train_loader=DataLoader(TensorDataset(x_train_t,y_train_t),batch_size=64,shuffle=True)
  model=SeqModel(input_dim=x_train.shape[2],hidden_dim=64,rnn_type=rnn_type).to(device)
  criterion=nn.BCEWithLogitsLoss()
  optimizer=optim.Adam(model.parameters(),lr=0.001)
  for epoch in range(5):
    model.train()
    for xb,yb in train_loader:
      optimizer.zero_grad()
      preds=model(xb).squeeze()
      loss=criterion(preds,yb)
      loss.backward()
      optimizer.step()
    #validation
    model.eval()
    with torch.no_grad():
      val_preds=torch.sigmoid(model(x_val_t)).cpu().numpy()
      value_preds=(val_preds>0.5).astype(int)
      val_acc=(value_preds==y_val).mean()
    print(f"{rnn_type.upper()} Epoch {epoch+1}/5 - Val Accuracy: {val_acc:.3f}")
  return model

In [9]:
rnn_model=train_model(x_train,y_train,x_val,y_val,rnn_type="rnn")
lstm_model=train_model(x_train,y_train,x_val,y_val,rnn_type="lstm")
gru_model=train_model(x_train,y_train,x_val,y_val,rnn_type="gru")

RNN Epoch 1/5 - Val Accuracy: 0.576
RNN Epoch 2/5 - Val Accuracy: 0.427
RNN Epoch 3/5 - Val Accuracy: 0.425
RNN Epoch 4/5 - Val Accuracy: 0.425
RNN Epoch 5/5 - Val Accuracy: 0.423
LSTM Epoch 1/5 - Val Accuracy: 0.449
LSTM Epoch 2/5 - Val Accuracy: 0.422
LSTM Epoch 3/5 - Val Accuracy: 0.422
LSTM Epoch 4/5 - Val Accuracy: 0.418
LSTM Epoch 5/5 - Val Accuracy: 0.418
GRU Epoch 1/5 - Val Accuracy: 0.442
GRU Epoch 2/5 - Val Accuracy: 0.435
GRU Epoch 3/5 - Val Accuracy: 0.423
GRU Epoch 4/5 - Val Accuracy: 0.426
GRU Epoch 5/5 - Val Accuracy: 0.425


In [10]:
def evaluate(model,x_test,y_test):
  model.eval()
  x_test_t=torch.tensor(x_test,dtype=torch.float32).to(device)
  with torch.no_grad():
    preds=torch.sigmoid(model(x_test_t)).cpu().numpy()
    preds=(preds>0.5).astype(int)
  return (preds==y_test).mean()
print("RNN Test Acc:",evaluate(rnn_model,x_test,y_test))
print("LSTM Test Acc:",evaluate(lstm_model,x_test,y_test))
print("GRU Test Acc:",evaluate(gru_model,x_test,y_test))

RNN Test Acc: 0.46361058601134214
LSTM Test Acc: 0.46195652173913043
GRU Test Acc: 0.4818052930056711
